# Observational study 

In [15]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotnine
#from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy
import time
import datetime

In [16]:
from popularity import trending_history
from financials import *

In [17]:
elon_org_df = pd.read_csv('Data/org-lg-Elon Musk.csv.bz2')

## Matching : 

In [18]:
def compareCompanies(companies, tickers, date, shares, hist):
    # Returns a df with the following columns: ['Date', 'Close', 'Volume', 'MarketCap', 'Popularity']
    df = pd.DataFrame(companies, columns=['Name'])
    df['Date'] = date
    
    marketData = []

    for i in range(len(tickers)):
        marketData.append(obs_info(tickers[i], date, shares[i], hist[i]).values.tolist())
    
    df['Close'] = [i[0] for i in marketData]
    df['Volume'] = [i[1] for i in marketData]
    df['MarketCap'] = [i[2] for i in marketData]
    
    df_trend = trending_history(companies, date)
    df_trend_list = df_trend.values.tolist()[0]
    df['Popularity'] = df_trend_list
    
    return df

def obs_info(ticker, date, sharesOutstanding, comp_hist):
    for i in range(len(comp_hist)):
        if comp_hist.iloc[i].name.date() > datetime.date.fromisoformat(date):
            df = comp_hist.iloc[i]
            break

    df['MarketCap'] = sharesOutstanding*df['Close']
    
    return df

In [19]:
#Data is our data frame with all the companies with first column 1 or 0 if Elon Musk talks about it.
# after,Name of the company and the other four covariates 
def compare(company_quote, date, companies, tickers, shares, hist): 

    company = compareCompanies(companies, tickers, date, shares, hist)

    company ['Elon']= 0
    company ['compare'] = company_quote
    company.at[4,'Elon'] = 1
    company.at[4,'compare'] = 'None'
    return company
    
#Call this function for all our functions and a set of company that will represent the controled one.

In [37]:
def create_data(companies_list, start, end):
    Company = companies_list
    company_quotes = Company [-1]
    quotes = elon_org_df[elon_org_df['ORG'] == company_quotes]
    first_date = quotes.iloc[0]['date'][0:10]
    ticker =[ticker_of_company(Company[0]),
             ticker_of_company(Company[1]),
             ticker_of_company(Company[2]),
             ticker_of_company(Company[3]),
             ticker_of_company(Company[4])] 
    shares=[yf.Ticker(ticker[0]).info['sharesOutstanding'],
            yf.Ticker(ticker[1]).info['sharesOutstanding'],
            yf.Ticker(ticker[2]).info['sharesOutstanding'],
            yf.Ticker(ticker[3]).info['sharesOutstanding'],
            yf.Ticker(ticker[4]).info['sharesOutstanding']]
    hist =[yf.Ticker(ticker[0]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[1]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[2]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[3]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[4]).history(start=first_date)[['Close', 'Volume']]]
    data = pd.DataFrame(columns = ['Name','Date','Close','Volume','MarketCap','Popularity','Elon','compare'])
    for i in range (start,end):
        date = quotes.iloc[i,:]['date'][0:10]
        company = compare(company_quotes, date, Company, ticker, shares, hist)
        frame = [data, company]
        data = pd.concat(frame)
    return data

In [25]:
#Apple: 
data_Apple = create_data(['Microsoft','IBM','Samsung','Dell','Apple'] )

<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on

ResponseError: The request failed: Google returned a response with code 429.

In [31]:
#PayPal : 
data_PayPal = create_data(['Western Union','EuroNet','MoneyGram','Payoneer','PayPal'])

<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
#Twitter : 
data_Twitter = create_data(['Google','Facebook','Snapchat','Pinterest','Twitter'])

<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#Tesla: 
data_Tesla = create_data(['Rivian','Lucid','VolksWagen','General Motors','Tesla'],0,1000)

<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
#Ford : 
data_Ford = create_data(['Toyota','BMW','General Motors','Stellantis','Ford'])

<ipython-input-18-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_Tesla

In [7]:
frame = [data_Apple, data_Tesla,data_Twitter, data_Ford, data_PayPal]
data = pd.concat(frame)
data.to_csv('Data/FinalFeatures.csv.bz2' , index=False)

In [ ]:
#Propensity score : 
def add_propensityscore (companies, date):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/lalonde_data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='elon ~ Close+ Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [ ]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [ ]:
def matching(companies):
    treatment_df = data[data['elon'] == 1]
    control_df   = data[data['elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (controle_row['compare']== treatment_id) and (controle_row['date']== treatment_row['date']) : 
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [ ]:
matching = matching(data)
matched = [i[0] for i in list(matching)]+ [i[1]for in list(matching)]
balanced = data.iloc[matched]


## After matching : 


In [ ]:
before =time.time()
quotes_Apple = elon_org_df[elon_org_df['ORG'] == 'Apple']
first_date = quotes_Apple.iloc[0]['date'][0:10]
Company_Apple =['Microsoft','IBM','Samsung','Dell','Apple'] 
ticker_Apple =[ticker_of_company('Microsoft'),
               ticker_of_company('IBM'),
               ticker_of_company('Samsung'),
               ticker_of_company('Dell'),
              ticker_of_company('Apple')] 
shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
               yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
               yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]
after =time.time()
print('fetch time:',str(after-before))

In [ ]:
hist_Apple[0].iloc[0]

In [ ]:
hist_Apple[0].iloc[0].name

In [ ]:
hist_Apple[0].iloc[0].name.date() > datetime.date.fromisoformat('2015-12-04')

In [ ]:
for i in range(len(hist_Apple[0])):
    if hist_Apple[0].iloc[i].name.date() > datetime.date.fromisoformat('2015-12-12'):
        print(hist_Apple[0].iloc[i])
        break